### Objetivo
Desenvolver, treinar e avaliar modelos preditivos para estimar a probabilidade da variável target (y), utilizando os dados pré-processados no Notebook 2. A modelagem respeita a separação temporal dos dados e avalia o desempenho por meio de métricas adequadas a problemas de crédito.

In [2]:
import sys
import os

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(ROOT_DIR)

import pandas as pd
import numpy as np

from src.modelagem import treinar_e_avaliar
from src.utils import salvar_objeto


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns


#### #1 Importação dos dados
Nesta etapa são importados os datasets de treino, validação e teste previamente processados no Notebook 2, garantindo consistência e reprodutibilidade no fluxo de modelagem.

In [3]:
# Treino
df_treino_mediana_base = pd.read_csv('../data/processed/treino_mediana.csv')

# Validação
df_validacao_mediana_base = pd.read_csv('../data/processed/validacao_mediana.csv')

# Teste
df_teste_mediana_base = pd.read_csv('../data/processed/teste_mediana.csv')



#### #2 Treinamento do modelo
Nesta etapa é realizado o treinamento de um modelo base de Regressão Logística, escolhido por sua ampla utilização em problemas de credit scoring, devido à sua interpretabilidade, estabilidade e facilidade de governança.

O modelo foi treinado por meio de uma função dedicada, utilizando os seguintes parâmetros:

- target_col = "y"<br>
- max_iter = 2000<br>
- penalty = "l2"<br>
- random_state = 42<br>

target_col indica para a função qual a variavel resposta do modelo<br>
A regularização L2 foi adotada com o objetivo de reduzir overfitting<br> 
O parâmetro max_iter foi definido como 2000 para garantir a convergência do algoritmo, evitando interrupções prematuras do processo de otimização<br>
O uso de random_state = 42 assegura a reprodutibilidade dos resultados, permitindo que o treinamento do modelo produza os mesmos coeficientes em execuções futuras.<br><br>

Adicionalmente, a função desenvolvida já realiza o cálculo das métricas de desempenho e do Population Stability Index (PSI) para cada variável, bem como a extração dos coeficientes do modelo, auxiliando na comparação entre variáveis e em etapas de ajuste fino (fine tuning).

In [4]:
pipelines = {
    'Modelo_Base': (df_treino_mediana_base, df_validacao_mediana_base, df_teste_mediana_base)
}

# Chamada da função
resultados, modelos, psi = treinar_e_avaliar(
    pipelines=pipelines,
    target_col="y",
    max_iter=2000,
    random_state=42,
    penalty='l2',

)


Pipeline utilizada: Modelo_Base
              AUC      KS    Gini
Treino     0.8003  0.4449  0.6006
Validação  0.7499  0.3776  0.4997
Teste      0.7007  0.3148  0.4014


#### #3 Avaliação do modelo
Nesta etapa é realizada a análise dos coeficientes do modelo e do Population Stability Index (PSI) das variáveis, com o objetivo de apoiar o ajuste fino do modelo.

As variáveis que apresentam valores de PSI superiores a 0,20 são analisadas com maior atenção, uma vez que esse patamar pode indicar instabilidade temporal relevante. Nessas situações, avalia-se a possibilidade de exclusão da variável, considerando conjuntamente seu impacto no desempenho do modelo e sua importância preditiva.

In [5]:
psi['Modelo_Base']['variaveis']['treino_teste'].T

,VAR_53,VAR_54,VAR_30,VAR_1,VAR_9,VAR_20,VAR_4,VAR_65,VAR_5,VAR_3,...,VAR_72,VAR_6,VAR_2,VAR_58,VAR_32,VAR_24,VAR_28,VAR_40,VAR_64,VAR_19
PSI,1.058077,0.403143,0.387795,0.216453,0.029952,0.027673,0.023268,0.015177,0.012540,0.011457,...,0.008041,0.007865,0.005983,0.005762,0.005728,0.005676,0.003404,0.003305,0.001920,0.001850
coeficiente,-0.219085,-0.106619,-0.237186,-1.096325,-1.016982,-0.763449,-0.015128,-0.161543,0.119927,0.212895,...,0.402682,-0.142722,0.470856,-0.156270,-0.352927,-0.635240,-0.048598,0.014576,0.218992,-0.182664


A avaliação conjunta do PSI e dos coeficientes do modelo permitiu as seguintes conclusões:<br>

Avaliando cada uma delas pelo PSI e coeficiente, podemos concluir<br>
- VAR_53: apresentou PSI muito elevado, indicando uma mudança significativa na população entre as bases de treino e teste. Dado o alto nível de instabilidade, o critério adotado foi a remoção da variável.<br>
- VAR_54: apresentou PSI elevado, indicando instabilidade temporal, além de baixa relevância no modelo. Dessa forma, optou-se pela remoção da variável.<br>
- VAR_30: apresentou comportamento semelhante, com PSI elevado porém com uma maior relevância para o modelo. Nesse caso o critério adotado foi a manutenção da variável no modelo, com recomendação de monitoramento contínuo em produção.<br>
- VAR_1: apresentou PSI levemente elevado, porém com forte relevância preditiva. Nesse caso, o critério adotado foi também a manutenção da variável no modelo, com recomendação de monitoramento contínuo em produção.<br>


#### #4 Ajuste fino do modelo
Nesta etapa é realizado o ajuste fino do modelo por meio da remoção das variáveis previamente identificadas como instáveis ou pouco relevantes, com o objetivo de melhorar a robustez, a estabilidade temporal e a qualidade geral do modelo.

In [6]:
# Treino
df_treino_mediana_refinado = df_treino_mediana_base.drop(columns=['VAR_53', 'VAR_54'])

# Validação
df_validacao_mediana_refinado = df_validacao_mediana_base.drop(columns=['VAR_53', 'VAR_54'])

# # Teste
df_teste_mediana_refinado = df_teste_mediana_base.drop(columns=['VAR_53', 'VAR_54'])

In [7]:
pipelines = {
    'Modelo_Refinado': (df_treino_mediana_refinado, df_validacao_mediana_refinado, df_teste_mediana_refinado)
}

# Chamada da função
resultados, modelos, psi = treinar_e_avaliar(
    pipelines=pipelines,
    target_col="y",
    max_iter=2000,
    random_state=42,
    penalty='l2',
    resultados_existentes=resultados,
    modelos_existentes=modelos,
    psi_existente=psi

)


Pipeline utilizada: Modelo_Refinado
              AUC      KS    Gini
Treino     0.8000  0.4383  0.5999
Validação  0.7515  0.3823  0.5030
Teste      0.7009  0.3067  0.4019


In [8]:
psi['Modelo_Refinado']['variaveis']['treino_val'].T

,VAR_30,VAR_1,VAR_65,VAR_20,VAR_3,VAR_9,VAR_72,VAR_2,VAR_5,VAR_58,VAR_6,VAR_4,VAR_24,VAR_28,VAR_32,VAR_57,VAR_40,VAR_19,VAR_64
PSI,0.241114,0.196292,0.019512,0.015894,0.013605,0.012821,0.009238,0.00906,0.006748,0.006087,0.005812,0.005563,0.004368,0.002145,0.001579,0.001299,0.000682,0.000676,0.000278
coeficiente,-0.303786,-1.101242,-0.147753,-0.774815,0.214714,-1.116352,0.399509,0.46355,0.120069,-0.154174,-0.143432,-0.016828,-0.624055,-0.048168,-0.359303,-0.014723,0.014385,-0.188700,0.220917


#### #5 Salvamento de dados
Nesta etapa são salvos os modelos treinados e os principais resultados obtidos, garantindo reprodutibilidade, rastreabilidade e consistência para as análises subsequentes. Esses artefatos serão utilizados em um notebook dedicado à comparação visual de desempenho e estabilidade entre modelos.

In [9]:
# Modelos
salvar_objeto(modelos, "../models/modelos.pkl")

# Resultados Métricas
salvar_objeto(resultados, "../artifacts/resultados.pkl")

# Resultados PSI
salvar_objeto(psi, "../artifacts/psi.pkl")

#### #6 Considerações finais - Modelagem
Neste notebook foi desenvolvido e avaliado um modelo de credit scoring com foco não apenas em performance preditiva, mas principalmente em estabilidade e robustez, aspectos fundamentais para aplicações em risco de crédito.

Inicialmente, foi treinado um modelo base de Regressão Logística, utilizando imputação por mediana e avaliado nas bases de treino, validação e teste por meio de métricas tradicionais como AUC e KS. Em paralelo, foi conduzida uma análise detalhada de Population Stability Index (PSI), tanto por variável quanto para o score final, permitindo avaliar possíveis mudanças de distribuição entre as populações.

A análise de PSI por variável evidenciou a presença de drift relevante em algumas features, especialmente aquelas com PSI elevado e baixo impacto no modelo, conforme indicado pelos coeficientes da regressão. Com base nesses critérios, foi realizado um processo de fine tuning, removendo variáveis instáveis e re-treinando o modelo.

O modelo refinado apresentou:

Leve redução nas métricas de performance (AUC e KS), dentro de limites aceitáveis;

Redução do PSI por variável, indicando maior estabilidade;

Pequena melhora no PSI do score, reforçando a robustez do modelo frente a mudanças de população.

Esses resultados reforçam um ponto fundamental em modelagem de crédito: a busca pelo melhor equilíbrio entre performance e estabilidade. Um modelo com ligeiramente menor poder discriminatório, mas mais estável, tende a apresentar melhor desempenho em produção e menor risco de degradação ao longo do tempo.